<h1 style="text-align: center;">Azure DevOps Comments Monitor</h1>
<p style="text-align: center;"><em>Powered by Google ADK &amp; Azure DevOps MCP</em></p>

**This agent automatically monitors Azure DevOps work items for mentions of specific team members and delivers daily email summaries.** 

The agent works with any Azure DevOps organization and is perfect for engineering managers, team leads, and individual contributors who need to stay informed about discussions, decisions, and action items where they're mentioned.

### How It Works

1. **Query Work Items** - Retrieves work items from a saved Azure DevOps query
2. **Fetch Comments** - Collects all comments from each work item
3. **Filter & Process** - Identifies mentions of the target user within the last 2 days
4. **Format Report** - Creates a structured summary with work item context and full comment text
5. **Send Notification** - Delivers the formatted report via email


### Prerequisites

To run the Azure DevOps Mentions Monitor agent with an MCP connection, you need:

* **Python 3.10+** with a virtual environment (venv)
* **JupyterLab** (installed inside the venv) or Google Colab
* **Node.js** - includes npx, which is used to launch the Azure DevOps MCP server (@azure-devops/mcp) on the fly without a global install
* **Azure DevOps Access** - Valid Personal Access Token (PAT) with Work Items (Read) permissions
* **Email Credentials** - Gmail account with App Password for sending notifications
* **Environment Variables** configured in `.env` file:
  * `GOOGLE_API_KEY` - Gemini API key for the agent's LLM
  * `EMAIL_USER` - Gmail address for sending notifications
  * `EMAIL_PASS` - Gmail App Password
  * `TO_EMAIL` - Recipient email address for summaries


### Customization

You can easily customize the agent by modifying:

* **Target User** - Change `@Jenya Stoeva` to any user's display name
* **Time Window** - Adjust the 2-day filter to any desired timeframe
* **Query ID** - Point to different saved queries in your Azure DevOps organization
* **Email Format** - Modify the email template and subject line
* **Organization/Project** - Configure for any Azure DevOps organization and project

In [ ]:
%pip install google-adk mcp

In [1]:
import asyncio
import contextlib
import os
from datetime import datetime
from typing import Optional

from dotenv import load_dotenv
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset, StdioConnectionParams, StdioServerParameters
from google.adk.tools.tool_context import ToolContext

In [2]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")
to_email = os.getenv("TO_EMAIL")  # Your email address
from_email = os.getenv("EMAIL_USER")  # Gmail address
password = os.getenv("EMAIL_PASS")  # Gmail app password

if GEMINI_API_KEY:
    print("Google API key loaded!")
else:
    print("Google API key missing!")

Google API key loaded!


In [3]:
import smtplib
from email.mime.text import MIMEText

def send_devops_comments_summary_email(
    summary: str,
    tool_context: Optional[ToolContext] = None
) -> str:
    """Send email notification with Azure DevOps mentions summary."""
    
    if not all([from_email, password, to_email]):
        return "❌ Email credentials not configured. Set EMAIL_USER, EMAIL_PASS, and TO_EMAIL."
    
    try:
        current_date = datetime.now().strftime("%Y-%m-%d %H:%M")
        subject = f"[Azure DevOps] Daily Mentions Summary - {current_date}"
        
        body = f"""
Azure DevOps Mentions Summary
{'=' * 60}
Generated: {current_date}
            
{summary}
            
{'=' * 60}
This is an automated notification from your Azure DevOps Assistant.
            """
        
        msg = MIMEText(body, "plain")
        msg["Subject"] = subject
        msg["From"] = from_email
        msg["To"] = to_email
        
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
            smtp.login(from_email, password)
            smtp.send_message(msg)
        
        return f"✅ Email sent to {to_email}"
        
    except Exception as e:
        return f"❌ Failed to send email: {str(e)}"

In [4]:
agent = Agent(
    name="azure_devops_assistant",
    model="gemini-2.5-flash",
    description="Azure DevOps assistant for monitoring specific person mentions in work items",
    instruction="""
        You are an Azure DevOps assistant that retrieves and summarizes recent comments where @Jenya Stoeva is mentioned.
        
        **TRICT WORKFLOW - FOLLOW THESE STEPS IN ORDER:**

        **Step 1:**  **Get work items from the saved query**
            - Call `wit_get_query_results_by_id` with:
              {
                "id": "2f7d034a-9d9d-4dce-97ff-b617811c82ac",
                "project": "MARMIND",
                "organization": "UPPER-Network"
              }
            - Extract all work item IDs from the query results
        
        **Step 2:**  **For each work item, retrieve all comments**
            - Use the tool: `wit_list_work_item_comments`
            - Pass the work item ID
            - Store all comments with their metadata (text, author, date)
        
        **Step 3:**  Filter Comments by Date and Mentions
            - ONLY INCLUDE comments from last 2 days (from {current_date})
            - ONLY INCLUDE comments mentioning "@Jenya Stoeva"
            - IGNORE all other comments, changes, or work item updates
        
            - For each included comment, capture:
              - Work Item ID
              - Work Item Title
              - Comment text (clean HTML, extract readable text)
              - Comment author
              - Comment date
              - Direct link to the work item

        **Step 4 (MANDATORY STEP - MUST ALWAYS SEND EMAIL):** Send Email Notification
        - Use the `send_devops_comments_summary_email` tool to send a summary to the user.
        - Pass the complete formatted output (both summary and detailed sections) as the `summary` parameter
        - Confirm email was sent successfully!
        
       **Summary FORMATTING RULES** 
       
        
            **If NO qualifying comments found:**
            Simply state: "No mentions of @Jenya Stoeva in comments from the last 2 days."
            
            **If qualifying comments ARE found, format as follows:**

            **Present Results in Two Sections**:
            
            ---
            
            📋 Summary of Recent Mentions (Last 2 Days). Total Work Items with Mentions: [X]
            
            📝 Work Item Summaries
            
            1️⃣. Work Item #[ID]: [Title]
            🔗 https://dev.azure.com/UPPER-Network/MARMIND/_workitems/edit/[ID]
            
            Summary: [2-3 sentence summary explaining what the comments are about, why Jenya was mentioned, and what action/decision/information was discussed]
            
            ---
            
            2️⃣. Work Item #[ID]: [Title]
            🔗 https://dev.azure.com/UPPER-Network/MARMIND/_workitems/edit/[ID]
            
            Summary: [2-3 sentence summary explaining what the comments are about, why Jenya was mentioned, and what action/decision/information was discussed]
            
            ---
            
            [Continue for ALL work items where Jenya was mentioned in comments from the last 2 days]
            
            ---
            
            📝 Detailed Comments (Full Text)
            
            1️⃣ Work Item #[ID]: [Title]
            🔗 https://dev.azure.com/UPPER-Network/MARMIND/_workitems/edit/[ID]
            
            Comment from [Author] on [Date]:
            > [Complete comment text with HTML cleaned - include the ENTIRE comment, not a summary]
            
            ---
            
            Comment from [Author] on [Date]:
            > [Complete comment text - if there are multiple comments mentioning Jenya in this work item]
            
            ---
            
            2️⃣ Work Item #[ID]: [Title]
            🔗 https://dev.azure.com/UPPER-Network/MARMIND/_workitems/edit/[ID]
            
            Comment from [Author] on [Date]:
            > [Complete comment text with HTML cleaned]
            
            ---
            
            [Continue for ALL qualifying comments in ALL work items where Jenya was mentioned]
            
            ---
        
        
        ## CRITICAL FORMATTING RULES:
        
        1. **Clean HTML**: Strip ALL HTML tags (<div>, <a>, <br>, etc.) and convert to readable text
        2. **Convert mentions**: Replace `<a href="#" data-vss-mention="...">@Name</a>` with plain `@Name`
        3. **Date filtering**: Comments MUST be from the last 48 hours
        4. **Complete comments**: In the detailed section, include FULL comment text, never truncate or summarize
        5. **Sort by date**: Show most recent work items/comments first
        6. **No duplicates**: Each work item should appear once in the summary section, but may have multiple comments in the detailed section
        
        ## IMPORTANT RULES:
        
        1. Complete Step 1 before moving to Step 2
        2. Retrieve comments for ALL work items before filtering
        3. Apply date filter: ONLY last 2 days (from {current_date})
        4. Apply mention filter: ONLY comments containing "@Jenya Stoeva"
        5. If a work item has comments but NONE mention "@Jenya Stoeva", EXCLUDE that work item entirely
        6. If a comment is older than 2 days, EXCLUDE it even if it mentions Jenya
        7. Summary section: Brief overview per work item with link
        8. Detailed section: Full text of ALL qualifying comments
        9. Both sections must include the SAME work items (consistent list)
        10. MUST ALWAYS send email with the complete formatted output
        
        ## ERROR HANDLING:
        
        - If Step 1 fails: Report the error and stop
        - If Step 2 fails for a specific work item: Note the failure, continue with other items
        - If date parsing fails for a comment: Include the comment with "Date unavailable"
        - If email sending fails: Report the error but still show the summary
        - If NO comments match criteria: Show "No mentions found" message and do NOT send email
        
        Begin execution immediately.
        """,
    tools=[
        send_devops_comments_summary_email,
        McpToolset(
            connection_params=StdioConnectionParams(
                server_params=StdioServerParameters(
                    command='npx',
                    args=[
                        '-y',
                        '@azure-devops/mcp',
                        'UPPER-Network'  # Your organization
                    ],
                ),
                timeout=120 
            )
        ),
    ],
)


# Setup session and runner  
session_service = InMemorySessionService()
runner = Runner(agent=agent, app_name="azure_devops_assistant", session_service=session_service)

# Global session setup
USER_ID = "user"
SESSION_ID = "azure_devops_conversation"
session_initialized = False


async def chat():
    """Start interactive conversation with the agent"""
    global session_initialized
    
    if not session_initialized:
        await session_service.create_session(
            app_name="azure_devops_assistant",
            user_id=USER_ID,
            session_id=SESSION_ID,
            state={'current_date': datetime.now().strftime("%Y-%m-%d")}  # Initialize state here!
        )
        
        session_initialized = True
        print("🤖 Hello! I'm your Azure DevOps assistant. Type 'quit', 'exit', 'stop' to end our chat.")
    
    
    while True:
        user_input = input("You: ").strip()
        
        if user_input.lower() in ['quit', 'exit', 'stop']:
            print("👋 Goodbye!")
            break
            
        if not user_input:
            continue

        # Update current_date in session state before each run
        session = await session_service.get_session(
            app_name="azure_devops_assistant",
            user_id=USER_ID,
            session_id=SESSION_ID
        )
        session.state['current_date'] = datetime.now().strftime("%Y-%m-%d")
            
        message = types.Content(role='user', parts=[types.Part(text=user_input)])
        print("Agent: ", end="", flush=True)
        
        with open(os.devnull, 'w') as f, contextlib.redirect_stderr(f):
            async for event in runner.run_async(
                user_id=USER_ID,
                session_id=SESSION_ID,
                new_message=message
            ):
                if event.content and event.content.parts:
                    for part in event.content.parts:
                        if part.text:
                            print(part.text, end="", flush=True)
        
        print("\n")

In [ ]:
await chat()